In [1]:
!pip install requests pandas pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.5 MB/s eta 0:00:00


In [2]:
import requests
import pandas as pd
import json

In [3]:
url = "https://services.cancerimagingarchive.net/services/v4/TCIA/query/getCollectionValues"
response = requests.get(url)

if response.status_code == 200:
    collections = response.json()
    df_collections = pd.DataFrame(collections)
    print(df_collections)
else:
    print("Error al acceder a la API:", response.status_code)

                      Collection
0                        4D-Lung
1               ACRIN-FLT-Breast
2            ACRIN-NSCLC-FDG-PET
3                         APOLLO
4                 Anti-PD-1_Lung
..                           ...
144                     CMB-BRCA
145        MIDI-B-Synthetic-Test
146  MIDI-B-Synthetic-Validation
147          MIDI-B-Curated-Test
148    MIDI-B-Curated-Validation

[149 rows x 1 columns]


In [4]:
# URL para obtener los estudios
studies_url = "https://services.cancerimagingarchive.net/services/v4/TCIA/query/getPatientStudy"

# Parámetros de la solicitud
params = {
    "Collection": "CMMD"
}

# Solicitud para obtener los estudios
response = requests.get(studies_url, params=params)

# Mostrar los estudios
if response.status_code == 200:
    studies = response.json()
    print(pd.DataFrame(studies))
else:
    print("Error al obtener los estudios:", response.status_code)

     Collection PatientID PatientName PatientSex  \
0          CMMD   D1-0860     D1-0860          F   
1          CMMD   D2-0140     D2-0140          F   
2          CMMD   D2-0205     D2-0205          F   
3          CMMD   D2-0070     D2-0070          F   
4          CMMD   D1-0577     D1-0577          F   
...         ...       ...         ...        ...   
1770       CMMD   D1-1713     D1-1713          F   
1771       CMMD   D1-0157     D1-0157          F   
1772       CMMD   D1-1057     D1-1057          F   
1773       CMMD   D1-0737     D1-0737          F   
1774       CMMD   D1-1696     D1-1696          F   

                                       StudyInstanceUID   StudyDate  \
0     1.3.6.1.4.1.14519.5.2.1.1239.1759.339893805217...  2010-07-18   
1     1.3.6.1.4.1.14519.5.2.1.1239.1759.598888059044...  2010-07-18   
2     1.3.6.1.4.1.14519.5.2.1.1239.1759.101522599910...  2010-07-18   
3     1.3.6.1.4.1.14519.5.2.1.1239.1759.305656210889...  2011-07-18   
4     1.3.6.1.4.1.14

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%ls /content/drive/MyDrive/Colab_Notebooks/TFM/CMMD.zip

/content/drive/MyDrive/Colab_Notebooks/TFM/CMMD.zip


In [7]:
!cp /content/drive/MyDrive/Colab_Notebooks/TFM/CMMD.zip /content/CMMD.zip


In [8]:
!unzip -q /content/CMMD.zip -d /content/CMMD


In [9]:
!find /content/CMMD -type f | wc -l


5203


In [10]:
# Cargar el clinical dataset
import pandas as pd

# Cargar el archivo Excel
density_df = pd.read_excel("/content/drive/MyDrive/Colab_Notebooks/TFM/density.xlsx")

lesion_df = pd.read_excel(
    "/content/drive/MyDrive/Colab_Notebooks/TFM/density.xlsx")

In [13]:
import pandas as pd
import pydicom
import os


# Directorio donde están las imágenes
dicom_dir = "/content/CMMD"

# Recoger los PatientID de las imágenes
image_data = []
for root, _, files in os.walk(dicom_dir):
    for file in files:
        if file.endswith(".dcm"):
            file_path = os.path.join(root, file)
            dicom_data = pydicom.dcmread(file_path)
            image_data.append({
                "ID": dicom_data.PatientID,
                "file_path": file_path
            })

# Crear un DataFrame con las imágenes
df_images = pd.DataFrame(image_data)

# Unir los datos clínicos con los datos de imagen
merged_data = pd.merge(df_images, density_df, on='ID', how='inner')

# Mostrar las primeras filas del conjunto de datos combinado
print(merged_data.head())

        ID                                          file_path LeftRight   Age  \
0  D2-0502  /content/CMMD/CMMD/D2-0502/07-18-2010-NA-NA-29...         L  83.0   
1  D2-0502  /content/CMMD/CMMD/D2-0502/07-18-2010-NA-NA-29...         R   NaN   
2  D2-0502  /content/CMMD/CMMD/D2-0502/07-18-2010-NA-NA-29...         L  83.0   
3  D2-0502  /content/CMMD/CMMD/D2-0502/07-18-2010-NA-NA-29...         R   NaN   
4  D2-0502  /content/CMMD/CMMD/D2-0502/07-18-2010-NA-NA-29...         L  83.0   

  classification Unnamed: 4       Breast density Mass  Unnamed: 7  Unnamed: 8  \
0      Malignant        NaN  heterogeneous dense    U  round/oval  Spiculated   
1         Normal        NaN  heterogeneous dense  NaN         NaN         NaN   
2      Malignant        NaN  heterogeneous dense    U  round/oval  Spiculated   
3         Normal        NaN  heterogeneous dense  NaN         NaN         NaN   
4      Malignant        NaN  heterogeneous dense    U  round/oval  Spiculated   

   ... Mass - Additional L

In [14]:
# Filtrar las filas que tengan clasificación válida
filtered_data = merged_data[merged_data['classification'].isin(['Benign', 'Malignant'])]
print(filtered_data.head())


        ID                                          file_path LeftRight   Age  \
0  D2-0502  /content/CMMD/CMMD/D2-0502/07-18-2010-NA-NA-29...         L  83.0   
2  D2-0502  /content/CMMD/CMMD/D2-0502/07-18-2010-NA-NA-29...         L  83.0   
4  D2-0502  /content/CMMD/CMMD/D2-0502/07-18-2010-NA-NA-29...         L  83.0   
6  D2-0502  /content/CMMD/CMMD/D2-0502/07-18-2010-NA-NA-29...         L  83.0   
8  D2-0575  /content/CMMD/CMMD/D2-0575/07-18-2011-NA-NA-11...         L  56.0   

  classification Unnamed: 4       Breast density Mass  Unnamed: 7  Unnamed: 8  \
0      Malignant        NaN  heterogeneous dense    U  round/oval  Spiculated   
2      Malignant        NaN  heterogeneous dense    U  round/oval  Spiculated   
4      Malignant        NaN  heterogeneous dense    U  round/oval  Spiculated   
6      Malignant        NaN  heterogeneous dense    U  round/oval  Spiculated   
8      Malignant        NaN  heterogeneous dense  M-L   irregular  Indistinct   

   ... Mass - Additional L

In [15]:
merged_data.shape

(8506, 34)

In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array

# Función para preprocesar imágenes
def preprocess_image(file_path):
    dicom_data = pydicom.dcmread(file_path)
    image = dicom_data.pixel_array.astype(np.float32)
    image = image / np.max(image)  # Normalización entre 0 y 1
    image = img_to_array(image)
    image = tf.image.resize(image, [256, 256])  # Redimensionar a 256x256
    return image

# Aplicar la función a las imágenes del dataset filtrado
filtered_data['processed_image'] = filtered_data['file_path'].apply(preprocess_image)


<ipython-input-16-e971740aa646>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['processed_image'] = filtered_data['file_path'].apply(preprocess_image)


In [17]:
from sklearn.model_selection import train_test_split

# Convertir las imágenes y etiquetas a arrays de NumPy
X = np.array(filtered_data['processed_image'].tolist())
y = (filtered_data['classification'] == 'Malignant').astype(int)  # 1: Malignant, 0: Benign

# Dividir los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [18]:
from tensorflow.keras import layers, models

# Definir el modelo
model = models.Sequential([
    layers.InputLayer(input_shape=(256, 256, 1)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), batch_size=16)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/5
213/213 ━━━━━━━━━━━━━━━━━━━━ 522s 2s/step - accuracy: 0.8486 - loss: 0.4573 - val_accuracy: 0.8834 - val_loss: 0.3759
Epoch 2/5
213/213 ━━━━━━━━━━━━━━━━━━━━ 554s 2s/step - accuracy: 0.8605 - loss: 0.3944 - val_accuracy: 0.8834 - val_loss: 0.3516
Epoch 3/5
213/213 ━━━━━━━━━━━━━━━━━━━━ 562s 2s/step - accuracy: 0.8635 - loss: 0.3558 - val_accuracy: 0.8834 - val_loss: 0.3742
Epoch 4/5
213/213 ━━━━━━━━━━━━━━━━━━━━ 565s 2s/step - accuracy: 0.8760 - loss: 0.2673 - val_accuracy: 0.8681 - val_loss: 0.4090
Epoch 5/5
213/213 ━━━━━━━━━━━━━━━━━━━━ 558s 2s/step - accuracy: 0.9138 - loss: 0.1683 - val_accuracy: 0.8693 - val_loss: 0.4330


In [19]:
# Evaluar el modelo
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Precisión en validación: {val_acc * 100:.2f}%")


27/27 ━━━━━━━━━━━━━━━━━━━━ 26s 958ms/step - accuracy: 0.8738 - loss: 0.4494
Precisión en validación: 86.93%


In [21]:
from sklearn.metrics import confusion_matrix

# Calcular la matriz de confusión
tn, fp, fn, tp = confusion_matrix(y_val, y_pred_bin).ravel()

print(f"True Positives (TP): {tp}")
print(f"False Positives (FP): {fp}")
print(f"True Negatives (TN): {tn}")
print(f"False Negatives (FN): {fn}")


True Positives (TP): 721
False Positives (FP): 82
True Negatives (TN): 17
False Negatives (FN): 29
